# Data Analysis : Clustering

### - Make caculated columns

In [2]:
import pandas as pd
import numpy as np

In [272]:
float_ppl = pd.read_csv('float_people_revised.csv', index_col=0)
resid_ppl = pd.read_csv('resid_people_revised.csv', index_col=0)

In [273]:
float_ppl

,시,구,행정동,년,월,시간대,성별,연령대,유동인구
0,대전광역시,동구,중앙동,2019,1,0,남,10대 미만,138
1,대전광역시,동구,중앙동,2019,1,0,남,10대,2518
2,대전광역시,동구,중앙동,2019,1,0,남,20대,18576
3,대전광역시,동구,중앙동,2019,1,0,남,30대,13751
4,대전광역시,동구,중앙동,2019,1,0,남,40대,16533
...,...,...,...,...,...,...,...,...,...
455035,대전광역시,대덕구,법2동,2020,3,23,여,30대,4518
455036,대전광역시,대덕구,법2동,2020,3,23,여,40대,4187
455037,대전광역시,대덕구,법2동,2020,3,23,여,50대,6225
455038,대전광역시,대덕구,법2동,2020,3,23,여,60대,2019


In [274]:
df = pd.merge(float_ppl, resid_ppl, how='outer')

In [278]:
df.replace(' - ', 0, inplace=True)

In [279]:
df['유동인구'] = df['유동인구'].astype('int')

In [87]:
df_12 = df[df['시간대']==12]

In [97]:
df_fin = df_12.groupby('행정동')['유동인구', '거주인구'].sum()

In [98]:
df_fin['유동-거주인구비율_12시'] = (df_fin['유동인구'] / df_fin['거주인구']) * 5760

In [110]:
df_fin.to_csv('유동-거주인구비율_12시.csv')

### - Clustering

In [1]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [20]:
df_cluster = pd.read_csv('clustering_v3.csv')

In [21]:
drop_col = ['유동인구', '거주인구', '10대 미만 합', '전체 합', 
            '10대', '10대 미만', '20대', '30대', '40대', '50대', '60대', '70대 이상'
           ,'유동-거주인구비율_12시', '속한 노드수']
df_cluster = df_cluster.drop(columns=drop_col)

In [22]:
df_cluster

,행정동,10대미만의 비율,near_phar,평균 나이
0,가수원동,0.166412,1.008002,37
1,가양1동,0.076700,2.773472,45
2,가양2동,0.071269,2.467993,43
3,가장동,0.092326,0.362630,43
4,갈마1동,0.076424,3.860900,41
...,...,...,...,...
74,판암1동,0.089477,1.143114,45
75,판암2동,0.042689,1.031373,52
76,홍도동,0.074769,2.012111,44
77,회덕동,0.072008,0.725398,46


In [23]:
X = df_cluster.drop(columns=['행정동'])
y = df_cluster['행정동']

In [24]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_pred = kmeans.fit_predict(X)

In [21]:
df_cluster['result'] = y_pred
df_cluster

,행정동,속한 노드수,10대미만의 비율,near_phar,평균 나이,result
0,가수원동,16.0,0.166412,1.008002,37,3
1,가양1동,6.0,0.076700,2.773472,45,2
2,가양2동,4.0,0.071269,2.467993,43,0
3,가장동,1.0,0.092326,0.362630,43,0
4,갈마1동,3.0,0.076424,3.860900,41,0
...,...,...,...,...,...,...
74,판암1동,5.0,0.089477,1.143114,45,2
75,판암2동,3.0,0.042689,1.031373,52,2
76,홍도동,2.0,0.074769,2.012111,44,2
77,회덕동,5.0,0.072008,0.725398,46,2


In [22]:
df_cluster[(df_cluster['행정동']=='신성동') | (df_cluster['행정동']=='관평동') | (df_cluster['행정동']=='전민동')]

,행정동,속한 노드수,10대미만의 비율,near_phar,평균 나이,result
8,관평동,30.0,0.142694,0.391419,36,1
46,신성동,36.0,0.148103,0.170223,36,1
65,전민동,17.0,0.126854,0.415469,37,3


- GMM

In [25]:
from sklearn.mixture import GaussianMixture
gmm = GaussianMixture(n_components = , n_init = 10, random_state=40)
gmm.fit(X)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=8, n_init=10,
                precisions_init=None, random_state=40, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [26]:
df_cluster['result'] = gmm.predict(X)

In [27]:
df_cluster[(df_cluster['행정동']=='신성동') | (df_cluster['행정동']=='관평동') | (df_cluster['행정동']=='전민동')]

,행정동,10대미만의 비율,near_phar,평균 나이,result
8,관평동,0.142694,0.391419,36,3
46,신성동,0.148103,0.170223,36,3
65,전민동,0.126854,0.415469,37,3


In [28]:
df_cluster.groupby('result').count()

,행정동,10대미만의 비율,near_phar,평균 나이
result,,,,
0,14,14,14,14
1,23,23,23,23
2,4,4,4,4
3,8,8,8,8
4,6,6,6,6
5,22,22,22,22
6,1,1,1,1
7,1,1,1,1


In [29]:
df_cluster[df_cluster['result']==3]

,행정동,10대미만의 비율,near_phar,평균 나이,result
0,가수원동,0.166412,1.008002,37,3
8,관평동,0.142694,0.391419,36,3
14,노은2동,0.128669,0.474740,38,3
15,노은3동,0.169331,1.130738,37,3
36,복수동,0.124685,0.723299,39,3
46,신성동,0.148103,0.170223,36,3
57,원신흥동,0.155358,0.844896,38,3
65,전민동,0.126854,0.415469,37,3


In [137]:
df_cluster.to_csv('clustering_gmm_v2.csv')